In [1]:
%matplotlib inline

import matplotlib.pyplot as plt

import sys
sys.path.append("/home/ubuntu/Notebooks/annsa/")

import tensorflow.contrib.eager as tfe
from sklearn import datasets, preprocessing, model_selection


import seaborn as sn
import annsa as an
import tensorflow as tf
import numpy as np

/home/ubuntu/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.python.ops.init_ops import glorot_uniform_initializer

In [3]:
tf.enable_eager_execution()

#### Define neural network

In [30]:
class simple_nn(tf.keras.Model):
    def __init__(self):
        super(simple_nn, self).__init__()
        """ Define here the layers used during the forward-pass 
            of the neural network.
        """
        l2_regularization_scale=1e-1
        nodes_layer_1=500
        nodes_layer_2=200
        dropout_probability=0.5 
        
        # define l2 regularization
        self.regularizer = tf.keras.regularizers.l2(l=l2_regularization_scale)
        #tf.contrib.layers.l2_regularizer(scale=10.0,scope='simple_nn')
        
        # Hidden layer.
        self.dense_layer1 = tf.layers.Dense(nodes_layer_1, 
                                            activation=tf.nn.relu,
                                            kernel_initializer=glorot_uniform_initializer(),
                                            kernel_regularizer=self.regularizer)
        self.drop1 = tf.layers.Dropout(dropout_probability)
        self.dense_layer2 = tf.layers.Dense(nodes_layer_2,
                                            activation=tf.nn.relu,
                                            kernel_initializer=glorot_uniform_initializer(),
                                            kernel_regularizer=self.regularizer)
        self.drop2 = tf.layers.Dropout(dropout_probability)
        # Output layer. No activation.
        self.output_layer = tf.layers.Dense(32, activation=None)
        
        
        
    
    
    def predict_logits(self, input_data):
        """ Runs a forward-pass through the network.     
            Args:
                input_data: 2D tensor of shape (n_samples, n_features).   
            Returns:
                logits: unnormalized predictions.
        """
        # Reshape input data
        input_data = tf.reshape(input_data,[-1,1,1024])
        hidden_activation1 = self.dense_layer1(input_data)
        hidden_activation1_dropout = 
        
        hidden_activation2 = self.dense_layer2(hidden_activation1)
        logits = self.output_layer(hidden_activation2)
        
        return logits
    
    def predict(self, input_data):
        """ Runs a forward-pass through the network.     
            Args:
                input_data: 2D tensor of shape (n_samples, n_features).   
            Returns:
                logits: unnormalized predictions.
        """
        # Reshape input data
        input_data = tf.reshape(input_data,[-1,1,1024])
        hidden_activation1 = self.dense_layer1(input_data)
        hidden_activation2 = self.dense_layer2(hidden_activation1)
        logits = self.output_layer(hidden_activation2)
        outputs = tf.nn.softmax(logits)
        return outputs
    
    def loss_fn(self, input_data, target):
        """ Defines the loss function used during 
            training.         
        """
        logits = self.predict_logits(input_data)
        cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=target, logits=logits))
        l2_weights = [model.trainable_weights[i] for i in range(len(model.trainable_weights)) if i%2==0]
        l2_loss = tf.contrib.layers.apply_regularization(self.regularizer, l2_weights)
        loss = cross_entropy_loss+l2_loss
        return loss
    
    def grads_fn(self, input_data, target):
        """ Dynamically computes the gradients of the loss value
            with respect to the parameters of the model, in each
            forward pass.
        """
        with tfe.GradientTape() as tape:
            loss = self.loss_fn(input_data, target) 
        return tape.gradient(loss, self.variables)
    
    def fit(self, input_data, target, optimizer, num_epochs=500, verbose=50):
        """ Function to train the model, using the selected optimizer and
            for the desired number of epochs.
        """
        for i in range(num_epochs):
            grads = self.grads_fn(input_data, target)
            optimizer.apply_gradients(zip(grads, self.variables))
            if (i==0) | ((i+1)%verbose==0):
                print('Loss at epoch %d: %f' %(i+1, self.loss_fn(input_data, target).numpy()))
                
    
    def fit_batch(self, train_dataset, optimizer, num_epochs=50, verbose=50):
        """ Function to train the model, using the selected optimizer and
            for the desired number of epochs.
        """
        for i in range(num_epochs):
            for (input_data, target) in tfe.Iterator(train_dataset.shuffle(1000).batch(32)):
                grads = self.grads_fn(input_data, target)
                optimizer.apply_gradients(zip(grads, self.variables))
            if (i==0) | ((i+1)%verbose==0):
                print('Loss at epoch %d: %f' %(i+1, self.loss_fn(input_data, target).numpy()))             

#### Get some data

In [5]:
training_data = np.load('/home/ubuntu/Notebooks/GADRAS_ANN_work/Project_SORMA18/Dataset_Details/spectrum_data_1-simplex_1E6.npy')[:-10000]
training_keys = np.load('/home/ubuntu/Notebooks/GADRAS_ANN_work/Project_SORMA18/Dataset_Details/key_data_1-simplex_1E6.npy')[:-10000]

training_data = an.log_normalize(training_data)
training_keys = an.normalize_data(training_keys)


X_tensor = tf.constant(training_data[0:100])
#y_tensor = tf.reshape(tf.constant(training_keys),[-1,32,1])
y_tensor = tf.constant(training_keys[0:100])

In [6]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_tensor, y_tensor))

#### Define an optimizer, model, and train the network

In [31]:
optimizer = tf.train.AdamOptimizer(5e-1)
model = simple_nn()
model.fit_batch(train_dataset, optimizer, num_epochs=100, verbose=50)

Loss at epoch 1: 135567.405527
Loss at epoch 50: 142.032700
Loss at epoch 100: 23.485587


#### Check outputs

In [39]:
index = 6

logits_test = model.predict(training_data[index])

an.results2(training_keys[index],5)
print '\n'
an.results2(logits_test.numpy()[0][0],5)

('U233', 0.9613300859883692)
('Back_U', 0.01466275659824047)
('Back_K', 0.01446393955962026)
('Back_Th', 0.009543217853770068)
('U235', 0.0)


('Back_U', 0.17874386273951262)
('Back_Th', 0.1550665890994291)
('Back_K', 0.13608594721507095)
('Cs137', 0.05527810998808426)
('Ir192', 0.0510488632334397)


#### Save model

In [9]:
# Specify checkpoint directory
checkpoint_directory = './eager-save-test.ckpt'
# Create model checkpoint
checkpoint = tfe.Checkpoint(optimizer=optimizer,
                            model=model,
                            optimizer_step=tf.train.get_or_create_global_step())

In [10]:
# Save trained model
checkpoint.save_network_checkpoint(file_prefix=checkpoint_directory)

AttributeError: 'Checkpoint' object has no attribute 'save_network_checkpoint'

#### Restore Model

In [ ]:
saver.restore('/tmp/ckpt')

In [11]:
# Specify checkpoint directory
checkpoint_directory = './eager-save-test.ckpt'
# Create model checkpoint
checkpoint = tfe.Checkpoint(optimizer=optimizer,
                            model=model,
                            optimizer_step=tf.train.get_or_create_global_step())

In [12]:
# Restore model from latest chekpoint
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_directory))

#### Check Predictions

In [13]:
model.loss_fn(X_train, y_train)

<tf.Tensor: id=61732, shape=(), dtype=float64, numpy=0.5215111970901489>

In [14]:
logits_test = model.predict(X_test)

print(logits_test)

tf.Tensor(
[[-0.03277973  0.42256092]
 [-0.17492533  1.08905442]
 [-0.15110942 -1.01793212]
 [-0.08231696  1.05194264]
 [-0.34405381 -0.23703221]
 [-0.46417323 -0.72362408]
 [-0.07200287  0.23013546]
 [-0.09701915  1.09780864]
 [-0.10120163  1.05055332]
 [ 0.03282835 -1.1802948 ]
 [-0.15293705  0.83410911]
 [-0.12119496  1.18870982]
 [-0.08622217  1.06893659]
 [-0.33596162 -0.26602851]
 [-0.04915239  0.32396122]
 [-0.31852946 -0.04176902]
 [-0.49785829 -0.76155645]
 [-0.33915985 -0.16889622]
 [-0.08941315  0.23483927]
 [-0.07538378  1.0626397 ]], shape=(20, 2), dtype=float64)
